In [1]:
# %% [markdown]
# ## Classical–Quantum comparison helper cell
#
# Solves A x = b with the NumPyLinearSolver (classical)
# and with HHL (quantum), then prints the results in
# a uniform, easy‑to‑compare format.

# %%
import numpy as np
from qiskit_aer import AerSimulator                     # backend
from qiskit.quantum_info import Statevector             # to unwrap the HHL state
from numpy_linear_solver import NumPyLinearSolver       # classical reference
from hhl import HHL                                     # Qiskit‑based HHL implementation

# ----- problem definition ----------------------------------------------------
A = np.array([[1, -1/3],
              [-1/3, 1]], dtype=float)
b = np.array([1, 0], dtype=float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b / np.linalg.norm(b)           # renormalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend = AerSimulator()
hhl_solver = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b)

# Helper: extract the two amplitudes that encode |x⟩ from the simulated state
def extract_solution(result):
    """Return the real 2‑component solution vector encoded in result.state."""
    sv = Statevector(result.state).data
    # For 1 work qubit, 2 phase‑est. qubits, 1 ancilla:
    # amplitudes |10000⟩ (decimal 16) and |10001⟩ (17) hold x₁ and x₂
    amp = sv[16:18].real
    return result.euclidean_norm * amp / np.linalg.norm(amp)

x_classical = classical_res.state                    # already a NumPy vector
x_quantum   = extract_solution(quantum_res)          # recovered from circuit

# ----- print side‑by‑side comparison -----------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂‑norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Classical solution vector: [1.125 0.375]
Quantum   solution vector: [1.11084849 0.37028283]

Classical Euclidean norm : 1.1858541225631423
Quantum   Euclidean norm : 1.1709371246996996

ℓ₂‑norm of (x_classical − x_quantum): 0.014916997863442845


In [2]:
# %% [markdown]
# ## Classical–Quantum comparison (4 × 4 linear system)
#
# We now solve a 4×4 tridiagonal Toeplitz system with both
# NumPyLinearSolver (classical reference) and HHL (quantum),
# then print the two solution vectors, their Euclidean norms
# and the ℓ₂‑distance between them.

# %%
import numpy as np
from scipy.sparse import diags
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from numpy_linear_solver import NumPyLinearSolver   # classical
from hhl                   import HHL              # quantum HHL

# ----- problem definition ----------------------------------------------------
NUM_WORK_QUBITS = 2                 # log₂(4) → 4‑dimensional vector
DIM              = 2 ** NUM_WORK_QUBITS

a_diag = 1
b_off  = -1/3
A = diags([b_off, a_diag, b_off], [-1, 0, 1],
          shape=(DIM, DIM)).toarray()
print("Matrix A:\n", A)
b_vec = np.array([1] + [0]*(DIM - 1), dtype=float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)     # re‑normalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    """Return the real 2ⁿ solution vector encoded in result.state.
       Assumes the (single) ancilla qubit is the MSB, and we take the
       amplitudes where ancilla = 1 and all phase‑est. qubits are 0."""
    sv            = Statevector(result.state).data
    total_qubits  = int(np.log2(len(sv)))
    base_index    = 1 << (total_qubits - 1)          # ancilla bit set to 1
    amps          = np.array([sv[base_index + i].real
                              for i in range(2 ** n_work_qubits)])
    # renormalise with the Euclidean norm returned by HHL
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ----- side‑by‑side printout -------------------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂‑norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Matrix A:
 [[ 1.         -0.33333333  0.          0.        ]
 [-0.33333333  1.         -0.33333333  0.        ]
 [ 0.         -0.33333333  1.         -0.33333333]
 [ 0.          0.         -0.33333333  1.        ]]
Classical solution vector: [1.14545455 0.43636364 0.16363636 0.05454545]
Quantum   solution vector: [1.11940309 0.44132129 0.16101556 0.08996872]

Classical Euclidean norm : 1.237833351044751
Quantum   Euclidean norm : 1.2173118517510284

ℓ₂‑norm of (x_classical − x_quantum): 0.04432756224443586


In [3]:
import numpy as np
from scipy.sparse import diags
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from numpy_linear_solver import NumPyLinearSolver   # classical
from hhl                   import HHL              # quantum HHL

# ----- problem definition ----------------------------------------------------
NUM_WORK_QUBITS = 4                 # log₂(4) → 4-dimensional vector
DIM              = 2 ** NUM_WORK_QUBITS

# Option 1: random REAL Hermitian (symmetric) matrix                # for reproducibility (optional)
M_real = np.random.randn(DIM, DIM)
A = (M_real + M_real.T) / 2         # A is now real symmetric

# Option 2: random COMPLEX Hermitian matrix
# np.random.seed(42)
# M_c = np.random.randn(DIM, DIM) + 1j*np.random.randn(DIM, DIM)
# A = (M_c + M_c.conj().T) / 2      # A is now complex Hermitian

print("Random Hermitian A:\n", A)

# right-hand side
b_vec = np.array([1] + [0]*(DIM - 1), dtype=complex if np.iscomplexobj(A) else float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)     # re-normalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    """Return the real (or complex) 2ⁿ solution vector encoded in result.state.
       Assumes the (single) ancilla qubit is the MSB, and we take the
       amplitudes where ancilla = 1 and all phase-est. qubits are 0."""
    sv            = Statevector(result.state).data
    total_qubits  = int(np.log2(len(sv)))
    base_index    = 1 << (total_qubits - 1)          # ancilla bit set to 1
    amps          = np.array([sv[base_index + i]
                              for i in range(2 ** n_work_qubits)])
    # renormalise with the Euclidean norm returned by HHL
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ----- side-by-side printout -------------------------------------------------
print("\nClassical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂-norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Random Hermitian A:
 [[ 2.35918461  0.80768914  0.04678012  1.11138356  1.36820949  2.11058586
   1.35621353  0.930286   -0.64991201  0.35299199  0.4006637   0.03596436
  -0.23126416 -0.04367758  0.56673173  0.81938656]
 [ 0.80768914  0.0436547  -1.20592971  0.23895785 -1.0337717   1.5047373
  -0.41623134 -0.24064239 -0.10500627  0.02380959  0.52187235 -0.13815977
  -0.04154049 -1.02900597 -0.54364541 -0.02269387]
 [ 0.04678012 -1.20592971 -0.12323398  0.4089494  -1.23598998  0.29672275
  -0.66195345  0.6461808  -0.70238813 -0.70818241 -0.15793517  0.83973102
   1.17961404 -0.59859559  1.1500576   0.35362276]
 [ 1.11138356  0.23895785  0.4089494   0.13966307  0.35208578 -0.29119943
   0.81019804 -1.00492483  0.82492192  0.89737629 -0.05807725  0.07840145
   0.06159903 -0.83792601  0.48120128  0.16305438]
 [ 1.36820949 -1.0337717  -1.23598998  0.35208578 -0.74547188  0.23036752
   1.26315029  0.59142963  0.19716512  0.10655602  0.59854937  0.78824733
  -0.65715964  0.16789429 -1.0307750

KeyboardInterrupt: 

In [ ]:
import numpy as np
from scipy.sparse import diags
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from numpy_linear_solver import NumPyLinearSolver   # classical
from hhl                   import HHL              # quantum HHL

# ----- problem definition ----------------------------------------------------
NUM_WORK_QUBITS = 3                 # log₂(4) → 4-dimensional vector
DIM              = 2 ** NUM_WORK_QUBITS

# Option 1: random REAL non-Hermitian (non-symmetric) matrix
# # for reproducibility (optional)
A = np.random.randn(DIM, DIM)       # pure random, no symmetrization

# Option 2: random COMPLEX non-Hermitian matrix
# np.random.seed(42)
# A = np.random.randn(DIM, DIM) + 1j*np.random.randn(DIM, DIM)
# (no Hermitian conjugation step)

print("Random non-Hermitian A:\n", A)

# right-hand side (use complex dtype if A is complex)
b_vec = np.array([1] + [0]*(DIM - 1),
                 dtype=complex if np.iscomplexobj(A) else float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)     # re-normalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    """Return the (possibly complex) 2ⁿ solution vector encoded in result.state.
       Assumes the (single) ancilla qubit is the MSB, and we take the
       amplitudes where ancilla = 1 and all phase-est. qubits are 0."""
    sv            = Statevector(result.state).data
    total_qubits  = int(np.log2(len(sv)))
    base_index    = 1 << (total_qubits - 1)          # ancilla bit set to 1
    amps          = np.array([sv[base_index + i]
                              for i in range(2 ** n_work_qubits)])
    # renormalise with the Euclidean norm returned by HHL
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ----- side-by-side printout -------------------------------------------------
print("\nClassical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂-norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Random non-Hermitian A:
 [[-0.01349722 -1.05771093  0.82254491 -1.22084365  0.2088636  -1.95967012
  -1.32818605  0.19686124]
 [ 0.73846658  0.17136828 -0.11564828 -0.3011037  -1.47852199 -0.71984421
  -0.46063877  1.05712223]
 [ 0.34361829 -1.76304016  0.32408397 -0.38508228 -0.676922    0.61167629
   1.03099952  0.93128012]
 [-0.83921752 -0.30921238  0.33126343  0.97554513 -0.47917424 -0.18565898
  -1.10633497 -1.19620662]
 [ 0.81252582  1.35624003 -0.07201012  1.0035329   0.36163603 -0.64511975
   0.36139561  1.53803657]
 [-0.03582604  1.56464366 -2.6197451   0.8219025   0.08704707 -0.29900735
   0.09176078 -1.98756891]
 [-0.21967189  0.35711257  1.47789404 -0.51827022 -0.8084936  -0.50175704
   0.91540212  0.32875111]
 [-0.5297602   0.51326743  0.09707755  0.96864499 -0.70205309 -0.32766215
  -0.39210815 -1.46351495]]

Classical solution vector: [ 0.27778144 -0.20569256  0.05735612 -0.00705174  0.16826868 -0.40106117
  0.0422284  -0.17579224]
Quantum   solution vector: [-0.0544329 